# NEW SECTION

In [ ]:
import dagshub
import datetime
import mlflow
import torch
import time
import os
import pandas as pd
from pathlib import Path
from roboflow import Roboflow
from ultralytics import YOLO, settings

In [ ]:
rf = Roboflow(api_key="aRTRw0ORiLGJdj7RF2gR")
project = rf.workspace("abiya-thesis").project("tennis-court-suuzy")
version = project.version(8)
dataset = version.download("yolov8", location="datasets/keypoints-v2")

In [ ]:
dagshub.init(repo_owner='abiyamf', repo_name='courtPlay', mlflow=True)
settings.update({"mlflow": True})

In [ ]:
# Global Config
os.environ["MLFLOW_KEEP_RUN_ACTIVE"] = "True"
time_now = datetime.datetime.now().strftime("%m-%d_%H-%M")
gpu_name = torch.cuda.get_device_name(0)
projectType = "key-points"
datasetVersion = "8"
yoloVersion = "yolo11"
modelSize = "l"
epochs = 1
imgsz = 1280
batch = 4
data_path = "datasets/keypoints-v2/data.yaml"
project_path = f"results/key-points-v2/{yoloVersion}/{modelSize}"

In [ ]:
def format_duration(seconds: float) -> str:
    """Convert seconds into 'Hh Mm Ss' format."""
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = int(seconds % 60)
    return f"{hours}h {minutes}m {secs}s"

def save_class_metrics(val_results):
    # Ambil nama class
    names = val_results.names

    # Data per-class
    data = {
        "class": [names[i] for i in range(len(names))],
        "precision": val_results.box.p.tolist(),
        "recall": val_results.box.r.tolist(),
        "mAP50": val_results.box.ap50.tolist(),
        "mAP50-95": val_results.box.maps.tolist(),
    }

    # Tambahkan baris agregat "all"
    all_row = {
        "class": "all",
        "precision": float(val_results.box.mp),
        "recall": float(val_results.box.mr),
        "mAP50": float(val_results.box.map50),
        "mAP50-95": float(val_results.box.map),
    }

    for key in data.keys():
        data[key].insert(0, all_row[key])

    # Buat DataFrame dan simpan
    df = pd.DataFrame(data)
    csv_path = os.path.join(str(val_results.save_dir), "class_metrics.csv")
    df.to_csv(csv_path, index=False)

    return csv_path

In [ ]:
# Load a model
model = YOLO(f"{yoloVersion}{modelSize}-pose.pt")

mlflow.set_experiment(f"courtPlay-{modelSize}")

start_time = time.time()

with mlflow.start_run(run_name=f"{time_now}"):
    mlflow.log_params({
        "project type": projectType,
        "dataset version": datasetVersion,
        "yolo version": yoloVersion,
        "model size": modelSize,
        "gpu": gpu_name,
        "epochs": epochs,
        "batch": batch,
        "imgsz": imgsz,
    })
    
    # Train the model
    results = model.train(
        data=data_path, 
        epochs=epochs, 
        imgsz=imgsz,
        batch=batch,    
        flipud=0.0,     # vertical flip
        fliplr=0.0,     # horizontal flip
        mosaic=0.0,     # mosaic augmentation
        mixup=0.0,      # mixup augmentation
        copy_paste=0.0,
        project=project_path,
        name=f"train/{time_now}",
        exist_ok=True
    )
    
    mlflow.log_params({
        "training time": format_duration(time.time() - start_time),
    })
    
    mlflow.log_artifact(save_class_metrics(results), artifact_path="class_metrics")

mlflow.end_run()

## Inference Image

In [ ]:
model = YOLO(f"{project_path}/train/{time_now}/weights/best.pt")
list_of_images = ["examples/images/"+i for i in os.listdir("examples/images/") if i.endswith(".jpg")]
results = model(list_of_images, stream=True)
os.makedirs(f"{project_path}/predict/{time_now}/", exist_ok=True)
for result in results:
    
    result.save(filename=f"{project_path}/predict/{time_now}/"+result.path.split("/")[-1])

## Inference Video

In [ ]:
model = YOLO(f"{project_path}/train/{time_now}/weights/best.pt")
list_of_videos = ["examples/videos/"+i for i in os.listdir("examples/videos/") if i.endswith(".mp4")]

# Make sure folder exists
os.makedirs(f"{project_path}/track/", exist_ok=True)

for video in list_of_videos:
    results = model.track(video, save=True, tracker="bytetrack.yaml", project=project_path, name=f"track/{time_now}/", exist_ok=True)